In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from math import nan
from datetime import datetime
import requests
import time
import json

# Get today's date
current_date = datetime.today()

In [2]:

driver = webdriver.Chrome()


In [3]:
from urllib.parse import urlsplit, urlunsplit

def remove_query_from_url(url):
    url_parts = urlsplit(url)
    url_without_query = urlunsplit((url_parts.scheme, url_parts.netloc, url_parts.path, '', ''))
    return url_without_query



In [17]:

path = "../../Documents/Ticket Sales.xlsx"
events = pd.read_excel(path, sheet_name ="Events")
events

,Artist,Date,Venue,Location,# of comments,Presale,Sellout Date,Sections,Min Cost,Max Resell,Notes
0,seven lions,2023-05-20,HISTORY,Toronto,35.0,2022-12-08,2023-02-07 00:00:00,NaN,NaN,NaN,NaN
1,gryffin,2023-06-01,REBEL,Toronto,23.0,2023-03-23,2023-05-31 00:00:00,NaN,NaN,NaN,NaN
2,Inquisitive,2023-07-29,ANNABELS,Toronto,14.0,2023-06-13,2023-07-20 00:00:00,NaN,NaN,NaN,NaN
3,veld 3 day 2023,2023-08-04,NaN,Toronto,NaN,2023-02-01,2023-07-28 00:00:00,NaN,269,319.0,NaN
4,veld 1 day 2023,2023-08-05,NaN,Toronto,NaN,2023-02-01,2023-07-28 00:00:00,NaN,129,219.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
362,Shakira,NaT,Scotiabank Arena,Toronto,NaN,NaT,NaN,NaN,0,306.0,NaN
363,dean lewis,NaT,History,Toronto,NaN,NaT,NaN,NaN,0,180.0,NaN
364,sabrina carpenter,NaT,Scotiabank Arena,Toronto,NaN,NaT,NaN,NaN,0,455.0,NaN
365,Hwasa,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [ ]:
def generate_stubhub_url(artist, location= "Toronto"):
    if pd.isna(artist):
        artist = "lol"
    if pd.isna(location):
        location = "Toronto"
    return "https://www.stubhub.ca/secure/search?q=" + artist + "%20" + location

def fetch_artist_data(artist):
    artist_search_url = generate_stubhub_url(artist)
    response = requests.get(artist_search_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    event_grid = soup.find('script', type='application/json', string=lambda x: x and 'eventGrids' in x)
    if event_grid is None:
        return None
    json_str = event_grid.text.strip()
    data = json.loads(json_str)
    return data

In [6]:
def close_prompts(driver):
    """
    Closes any modal that appears and applies ticket filters.
    """
    # Close any modal that appears
    try:
        driver.find_element(By.XPATH, '//*[@id="modal-root"]/div/div/div/div[2]/div[3]/button').click()
    except:
        pass
def apply_ticket_filters(driver):
    try:
        driver.find_element(By.CSS_SELECTOR, "div.sc-1urpwzu-1").click()
        reccomended_filter = driver.find_element(By.XPATH, "//*[@id='stubhub-event-detail-popular-filters']/div/div/div/div[2]/div/div/div/div[2]/div/input")
        if reccomended_filter.get_attribute("value") == "true":
            time.sleep(3)
            reccomended_filter.click()
    except:
        pass

def click_zones(driver):
    zones_div = driver.find_elements(By.CSS_SELECTOR, 'div.sc-1s9c4ms-2.jFxikH')
    for zone in zones_div:
        if "zones" in zone.text.lower():
            zone.click()
            break
    
def click_checkboxes(driver):
    """
    Clicks each checkbox, unchecks all others, and returns the link each time.
    """
    # Find and click the "Zones" button to expand the filter options

    checkboxes = driver.find_elements(By.CSS_SELECTOR, 'input[type="checkbox"].sc-mhai9k-2.fOnHmE')
    links = []

    for checkbox in checkboxes:
        # Uncheck all checkboxes first
        for cb in checkboxes:
            if cb.is_selected():
                cb.click()
        
        # Click the current checkbox
        checkbox.click()
        
        # Wait for the page to update
        time.sleep(2)
        
        # Get the current URL
        links.append(driver.current_url)
    
    return links

# Call the function and store the links



In [ ]:

import pandas as pd
from bs4 import BeautifulSoup
from sympy import symbols
import requests

events = pd.read_excel(path, sheet_name ="Events")


processed = []


new_df = pd.DataFrame()




for index, row in events.iterrows():

    # Get the page source
    if row["Artist"] in processed:
        continue

    if row["Artist"] not in ["black pink", 'Oasis']:
        continue


    processed.append(row["Artist"])

    artist_search_url = generate_stubhub_url(row["Artist"])
    response = requests.get(artist_search_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # print(soup.prettify())
    
    # Extract the JSON string from the script tag with id 'page-content'
    event_grid = soup.find('script', type='application/json',  string=lambda x: x and 'eventGrids' in x)

    if event_grid is None:
        continue
    json_str = event_grid.text.strip()
    data = json.loads(json_str)
    event_grid = data["eventGrids"]
    events =  event_grid["0"]["items"][0:10]
    for event in events:
        if event["countryName"] not in ["Canada", "USA"]:
            continue
    
        try:
            url = remove_query_from_url(event["url"]) + "?listingQty=&quantity=0"+"&betterValueTickets=false" + "&estimatedFees=false"
            driver.get(url)
            close_prompts(driver)
            time.sleep(1)
            apply_ticket_filters(driver)
            click_zones(driver)
            checkbox_links = click_checkboxes(driver)
        except:
            print(str(index) + " " + row["Artist"] + " error")
            checkbox_links = [url]
        for url in checkbox_links:

            # Get the page source using requests
            response = requests.get(url)
    
            page_source = response.text
            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(page_source, 'html.parser')
            # print(soup.prettify())

            # Find the script tag with id 'index-data'
            script_tag = soup.find('script', id='index-data', type='application/json')

            # Extract the JSON string from the script tag
            if script_tag is None:
                continue
            json_string = script_tag.string

            # Parse the JSON string
            index_data = json.loads(json_string)

            # Extract grid items
            grid_items = index_data['grid']['items']

            # Create a DataFrame from the grid items
            df = pd.DataFrame(grid_items)

            # Display the DataFrame        
            
            df["Venue"] = event["venueName"]
            df["Artist"] = row["Artist"]
            df["Event Name"] = event["name"]
            # Display the DataFrame
            df["City"] = event["venueCity"]
            df["Event Date"] = event['formattedDate']
            df["Event ID"] = event['eventId']
            df["countryName"] = event['countryName']
            df["Updated"] = pd.Timestamp.today().strftime('%Y-%m-%d')
        

            new_df = pd.concat([new_df, df], ignore_index=True, axis = 0)
  

new_df

In [ ]:

events = pd.read_excel(path, sheet_name ="Events")
unique_artists = events['Artist'].unique()




from enum import unique
import glob
import re
import time

import concurrent.futures

from sklearn.covariance import oas
from torch import res

def fetch_artist_data(artist):
    artist_search_url = generate_stubhub_url(artist)
    response = requests.get(artist_search_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    event_grid = soup.find('script', type='application/json', string=lambda x: x and 'eventGrids' in x)
    if event_grid is None:
        return None
    json_str = event_grid.text.strip()
    data = json.loads(json_str)
    event_grid = data["eventGrids"]
    events =  event_grid["0"]["items"][0:10]
    for event in events:
        event["aritst"] = artist
    return events



def get_listing_info(url):
    response = requests.get(url)
    
    page_source = response.text
    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find the script tag with id 'index-data'
    script_tag = soup.find('script', id='index-data', type='application/json')

    # Extract the JSON string from the script tag
    if script_tag is None:
        return None
    json_string = script_tag.string

    # Parse the JSON string
    index_data = json.loads(json_string)

    # Extract grid items
    grid_items = index_data['grid']['items']

    # Create a DataFrame from the grid items
    df = pd.DataFrame(grid_items)
    return df


combined_df = pd.DataFrame()

def get_checkbox_links(url):
    try:
        url = remove_query_from_url(url) + "?listingQty=&quantity=0"+"&betterValueTickets=false" + "&estimatedFees=false"
        driver = webdriver.Chrome()
        driver.get(url)
        close_prompts(driver)
        time.sleep(1)
        apply_ticket_filters(driver)
        click_zones(driver)
        checkbox_links = click_checkboxes(driver)
        driver.close()
    except:
        checkbox_links = [url]
    return checkbox_links

def attach_event_data(df, event):
    print(event)
    df["Venue"] = event["venueName"]
    df["Artist"] = event["artist"]
    df["Event Name"] = event["name"]
    df["City"] = event["venueCity"]
    df["Event Date"] = event['formattedDate']
    df["Event ID"] = event['eventId']
    df["countryName"] = event['countryName']
    df["Updated"] = pd.Timestamp.today().strftime('%Y-%m-%d')
    return df

def get_section_links(event):
    global combined_df
    if event["countryName"] not in ["Canada", "USA"]:
            return None
    checkbox_links = get_checkbox_links(event["url"])
    with concurrent.futures.ThreadPoolExecutor() as executor:
        listings = list(executor.map(get_listing_info, checkbox_links))
    curr = pd.DataFrame()
    for df in listings:
        if df is not None:
            curr = pd.concat([curr, df], ignore_index=True)
    curr = attach_event_data(curr, event)
       # Combine the DataFrame
    combined_df = pd.concat([combined_df, curr], ignore_index=True)

start_time = time.time()

unique_artists = ['black pink', 'oasis']
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(fetch_artist_data, unique_artists))


for r in results:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        list(executor.map(get_section_links, r))


combined_df

{'eventId': 157423305, 'queryId': '0839f488451359040bf493f2ac6361f4', 'invocationId': 'ca3a3f26-f8b9-49fe-87cb-91f0ce7ce364', 'userToken': '432d0876decd6b79481d4209af4a9afa81188c79', 'indexName': 'stubhub', 'name': 'BLACKPINK', 'url': 'https://www.stubhub.ca/blackpink-toronto-tickets-7-23-2025/event/157423305/?qid=0839f488451359040bf493f2ac6361f4&iid=ca3a3f26-f8b9-49fe-87cb-91f0ce7ce364&index=stubhub&ut=432d0876decd6b79481d4209af4a9afa81188c79', 'dayOfWeek': 'Wed', 'formattedDate': 'Jul 23', 'formattedTime': '8:00 PM', 'formattedDateWithoutYear': 'Jul 23', 'isTbd': False, 'isDateConfirmed': True, 'isTimeConfirmed': True, 'eventState': 1, 'venueId': 150158421, 'venueName': 'Rogers Stadium (Concert Venue, Canada)', 'formattedVenueLocation': 'Toronto, Canada', 'countryCode': 'CA', 'formattedMinPrice': '', 'userDistanceFromVenue': '11066.7219229435', 'formattedDistanceInMiles': '7 mi', 'formattedRescheduledFromDate': '', 'notificationSignupLinkMessage': {'message': 'Join the list', 'qualif

,id,clientApplicationId,eventId,section,sectionId,sectionMapName,sectionType,row,hasSeatDetails,hasSeatDetailsUS,...,soldXTimeAgoSiteMessage,Venue,Artist,Event Name,City,Event Date,Event ID,countryName,Updated,lastTicketInSectionMessage
0,8858218765,312,157423305,General Admission Pit,2068922,General Admission Pit,4,PIT1,False,False,...,NaN,"Rogers Stadium (Concert Venue, Canada)",BLACKPINK,BLACKPINK,Toronto,Jul 23,157423305,Canada,2025-02-28,NaN
1,8850621586,3,157423305,General Admission Pit,2068922,General Admission Pit,4,,False,False,...,NaN,"Rogers Stadium (Concert Venue, Canada)",BLACKPINK,BLACKPINK,Toronto,Jul 23,157423305,Canada,2025-02-28,NaN
2,8859306565,134,157423305,110,2018739,110,2,27,False,False,...,NaN,"Rogers Stadium (Concert Venue, Canada)",BLACKPINK,BLACKPINK,Toronto,Jul 23,157423305,Canada,2025-02-28,NaN
3,8859306894,134,157423305,117,2018746,117,2,18,False,False,...,NaN,"Rogers Stadium (Concert Venue, Canada)",BLACKPINK,BLACKPINK,Toronto,Jul 23,157423305,Canada,2025-02-28,NaN
4,8859347626,123,157423305,117,2018746,117,2,31,False,False,...,NaN,"Rogers Stadium (Concert Venue, Canada)",BLACKPINK,BLACKPINK,Toronto,Jul 23,157423305,Canada,2025-02-28,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,8829604307,1231,155283966,102 WCR,2018755,102 WCR,2,WC2,False,False,...,NaN,"Rogers Stadium (Concert Venue, Canada)",Oasis,Oasis,Toronto,Aug 24,155283966,Canada,2025-02-28,NaN
368,8829604301,1231,155283966,119 WCR,2018757,119 WCR,2,WC2,False,False,...,NaN,"Rogers Stadium (Concert Venue, Canada)",Oasis,Oasis,Toronto,Aug 24,155283966,Canada,2025-02-28,NaN
369,8829604305,1231,155283966,124 WCR,2018758,124 WCR,2,WC2,False,False,...,NaN,"Rogers Stadium (Concert Venue, Canada)",Oasis,Oasis,Toronto,Aug 24,155283966,Canada,2025-02-28,NaN
370,8048443106,654,155283966,102 WCR,2018755,102 WCR,2,WC1,False,False,...,NaN,"Rogers Stadium (Concert Venue, Canada)",Oasis,Oasis,Toronto,Aug 24,155283966,Canada,2025-02-28,NaN


In [9]:
asda d

SyntaxError: invalid syntax (756658157.py, line 1)

In [ ]:
xlsx_file = 'Concert Seats.xlsx'


df = pd.read_excel(xlsx_file)

# Merge the new grid items with the existing DataFrame
updated_df = pd.concat([df, new_df], axis=0).drop_duplicates(subset='id', keep='last')

# Save the updated DataFrame to the existing Excel file
with pd.ExcelWriter(xlsx_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    updated_df.to_excel(writer, sheet_name='Sheet1', index=False)

    

In [ ]:
updated_df

In [ ]:
seats = pd.read_excel('Concert Seats.xlsx', sheet_name='Sheet1')
short_seats =seats[['Artist', 'Venue','City', 'Event Date', 'section', 'row', 'faceValue', 'rawPrice']]




In [ ]:
with_face = short_seats[short_seats["faceValue"] >10]

In [ ]:
with_face['profit'] = with_face['rawPrice'] - with_face['faceValue']
with_face['margin'] = with_face['profit'] / with_face['faceValue']
with_face = with_face.sort_values(by = "margin", ascending = False,)


In [ ]:
rogers_stadium_stats = with_face[(with_face['Venue'] == 'Rogers Stadium') | (with_face['Venue'] == 'Rogers Stadium (Concert Venue, Canada)')]
rogers_stadium_stats

In [ ]:
# Calculate the IQR for each section and remove outliers
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Group by section and remove outliers
grouped = rogers_stadium_stats.groupby('section')
filtered_groups = [remove_outliers(group, 'margin') for name, group in grouped]
filtered_df = pd.concat(filtered_groups)

# Group by section and calculate statistics
rogers_stadium_grouped = filtered_df.groupby(['section']).agg({
    'margin': ['median', 'mean', 'min', 'count']
}).reset_index()

rogers_stadium_grouped.columns = ['section', 'median_margin', 'average_margin', 'min_margin', 'count']
rogers_stadium_grouped[rogers_stadium_grouped['count'] > 10].sort_values(by = "median_margin", ascending = False)

In [ ]:
rogers_stadium_stats = with_face[(with_face['Venue'] == 'Rogers Stadium') | (with_face['Venue'] == 'Rogers Stadium (Concert Venue, Canada)')]

rogers_stadium_grouped = rogers_stadium_stats.groupby(['Artist']).agg({
    'margin': ['median', 'mean', 'min', 'count']
}).reset_index()

rogers_stadium_grouped.columns = ['artist', 'median_margin', 'average_margin', 'min_margin', 'count']
rogers_stadium_grouped

In [ ]:
blackpink = rogers_stadium_stats[rogers_stadium_stats['Artist'] == 'black pink']
blackpink = blackpink.drop(columns=['Artist', 'Venue', 'City', 'Event Date'])
blackpink

In [ ]:
blackpink = rogers_stadium_stats[rogers_stadium_stats['Artist'] == 'black pink']

blackpink_stats = blackpink.groupby(['section',]).agg({
    'margin': ['median', 'mean', 'min' , 'count']
}).reset_index()

blackpink_stats.columns = ['section', 'median_price', 'average_price', 'min_price', 'count']
blackpink_stats

In [ ]:
with pd.ExcelWriter('Concert Seats.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    short_seats.to_excel(writer, sheet_name='Short Seats', index=False)